
# ViEWS 3 constituent models 
## ViEWS-ESCWA model production system, cm level


This notebook trains a set of regression models for use in the monthly updated ViEWS-ESCWA predicting  ensemble

The notebook does the following: 
1. Retrieves data through querysets and stores in DataSets, a list of dictionaries
2. Specifies the metadata of a number of models, stores in ModelList, a list of dictionaries
3. Trains the models in ModelList, stores the trained objects in model storage and prediction storage
4. Saves part of ModelList as csv and the rest as pickles


## Importing modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import XGBRFRegressor, XGBRFClassifier

from lightgbm import LGBMClassifier, LGBMRegressor

# Views 3
from viewser.operations import fetch
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_runs import storage
from views_runs.storage import store, retrieve, fetch_metadata

from views_forecasts.extensions import *

# Other packages
import pickle as pkl

# Packages from viewsforecasting repository

#from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated
import os
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
from FetchData import FetchData, RetrieveFromList, document_queryset, ReturnQsList, document_ensemble
from ViewsEstimators import *


## Common parameters

In [3]:
# Common parameters:
dev_id = 'escwa001'
run_id = dev_id

# Generating a new run if necessary

#try:
#    ViewsMetadata().new_run(name=run_id,description='Developing the fatalities model for FCDO',min_month=1,max_month=999)
#except KeyError:
#    if 'devel' not in run_id:
#        warnings.warn('You are overwriting a production system')

RerunQuerysets = True

EndOfHistory = 517
steps = [*range(1, 36+1, 1)] # Which steps to train and predict for
fi_steps = [1,3,6,12,36] # Which steps to present feature importances for
#steps = [1,3,6,12,36]
#fi_steps = [1,3,6,12,36]

# Specifying partitions
calib_partitioner_dict = {"train":(121,408),"predict":(409,456)}
test_partitioner_dict = {"train":(121,456),"predict":(457,504)}
future_partitioner_dict = {"train":(121,504),"predict":(505,516)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{os.getlogin()}/Dropbox (ViEWS)/ViEWS'
# Intitilaprint('Setting Mydropbox to',Mydropbox)

In [4]:
ViewsMetadata().get_runs()

,name,description,min_month,max_month
id,,,,
1,tests,Just for testing/development,NaN,NaN
2,devel,Development Runs,NaN,NaN
3,fat_2022_2,Fatalities run for February 2022,505.0,542.0
4,escwa_2022_2,ESCWA run for February 2022,505.0,542.0
5,jpr_2022_2,JPR run for February 2022,505.0,542.0
16,fat_devel_v1,Developing the fatalities model for FCDO,1.0,999.0
17,test_run_x1,Bug fixes and other problems,1.0,999.0
18,fat_devel_v2,Developing the fatalities model for FCDO,1.0,999.0
19,lags,Developing lag models,1.0,999.0


In [5]:
# Initializing the run:
initialize_run = False
if initialize_run:
    ViewsMetadata().new_run(name='escwa001', description='Respecified VIEWS-ESCWA model',min_month=1,max_month=999)

# Retrieve data

In [6]:
# Create Markdown documentation of all querysets used
level = 'cm'
qslist = ReturnQsList(level)
document_queryset(qslist,dev_id)

 .     .    escwa001_vdem A dataset with 52 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_baseline; A dataset with 6 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_topics_stub; A dataset with 64 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_aquastat_stub; A dataset with 11 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_cm_conflict_history_stub; A dataset with 24 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_cm_conflict_history_ext; A dataset with 33 columns, with data between t = 1 and 852. (213 units)
 .    fatalities002_vdem_short_stub; A dataset with 58 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_wdi_short_stub; A dataset with 28 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_joint_narrow; A dataset with 41 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_joint_broad_stub; A dataset

In [7]:
from FetchData import fetch_cm_data_from_model_def

Datasets=fetch_cm_data_from_model_def(qslist, EndOfHistory)

 .    vdem001: A dataset with 52 columns, with data between t = 1 and 852; 213 units.
 .    cflong001: A dataset with 44 columns, with data between t = 1 and 852; 213 units.


# Generating predictions
Using the ViEWS3 partitioning/stepshifting syntax. Training models for A: calibration partition and B: test partition, to test out some calibration routines. Most models trained with ln_ged_sb_best as outcome.

In [8]:
for ds in Datasets:
    if 'topics' in ds['Name']:
        print(ds['df'].columns)
    

## Checking missingness and infinity values

In [9]:
N=51
for i in range(len(Datasets)):
    df = Datasets[i]['df']
    print(Datasets[i]['Name'])
    for col in df.iloc[: , :N].columns:
        print(col,len(df[col]), 'missing:', df[col].isnull().sum(), 'infinity:', np.isinf(df).values.sum())


vdem001
ged_sb_dep 158230 missing: 0 infinity: 0
ged_sb_dummy_t0 158230 missing: 0 infinity: 0
vdem_v2x_delibdem 158230 missing: 15145 infinity: 0
vdem_v2x_egaldem 158230 missing: 15145 infinity: 0
vdem_v2x_libdem 158230 missing: 15145 infinity: 0
vdem_v2x_libdem_48 158230 missing: 15166 infinity: 0
vdem_v2x_partip 158230 missing: 15145 infinity: 0
vdem_v2x_partipdem 158230 missing: 15145 infinity: 0
vdem_v2x_accountability 158230 missing: 15145 infinity: 0
vdem_v2x_civlib 158230 missing: 15145 infinity: 0
vdem_v2x_clphy 158230 missing: 15145 infinity: 0
vdem_v2x_cspart 158230 missing: 15145 infinity: 0
vdem_v2x_divparctrl 158230 missing: 15145 infinity: 0
vdem_v2x_edcomp_thick 158230 missing: 15145 infinity: 0
vdem_v2x_egal 158230 missing: 15145 infinity: 0
vdem_v2x_execorr 158230 missing: 15145 infinity: 0
vdem_v2x_frassoc_thick 158230 missing: 15145 infinity: 0
vdem_v2x_gencs 158230 missing: 15145 infinity: 0
vdem_v2x_gender 158230 missing: 15391 infinity: 0
vdem_v2x_genpp 158230 mi

# Specify models in ensemble

In [10]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels('cm')
    
for imodel,model in enumerate(ModelList):
    print(imodel, model['modelname'], model['data_train'])

0 escwa001_cflong cflong001
1 escwa001_vdem vdem001


In [11]:
ModelList

[{'modelname': 'escwa001_cflong',
  'algorithm': RandomForestClassifier(n_estimators=300, n_jobs=12),
  'depvar': 'ged_sb_dep',
  'data_train': 'cflong001',
  'queryset': 'escwa001_cflong',
  'preprocessing': 'float_it',
  'level': 'cm',
  'description': 'Long conflict history model',
  'long_description': '...',
  'predstore_calib': 'cm_escwa001_cflong_calib',
  'predstore_test': 'cm_escwa001_cflong_test'},
 {'modelname': 'escwa001_vdem',
  'algorithm': RandomForestClassifier(n_estimators=300, n_jobs=12),
  'depvar': 'ged_sb_dep',
  'data_train': 'vdem001',
  'queryset': 'escwa001_vdem',
  'preprocessing': 'float_it',
  'level': 'cm',
  'description': 'Varieties of democracy model',
  'long_description': '...',
  'predstore_calib': 'cm_escwa001_vdem_calib',
  'predstore_test': 'cm_escwa001_vdem_test'}]

In [12]:
document_ensemble(ModelList,'sb')

0 escwa001_cflong cflong001
1 escwa001_vdem vdem001


In [13]:
for ds in Datasets:
    df = ds['df']
    print(ds['Name'],df.isna().sum())
    ds['df']=df.fillna(0)

vdem001 ged_sb_dep                     0
ged_sb_dummy_t0                0
vdem_v2x_delibdem          15145
vdem_v2x_egaldem           15145
vdem_v2x_libdem            15145
vdem_v2x_libdem_48         15166
vdem_v2x_partip            15145
vdem_v2x_partipdem         15145
vdem_v2x_accountability    15145
vdem_v2x_civlib            15145
vdem_v2x_clphy             15145
vdem_v2x_cspart            15145
vdem_v2x_divparctrl        15145
vdem_v2x_edcomp_thick      15145
vdem_v2x_egal              15145
vdem_v2x_execorr           15145
vdem_v2x_frassoc_thick     15145
vdem_v2x_gencs             15145
vdem_v2x_gender            15391
vdem_v2x_genpp             15391
vdem_v2x_horacc            15145
vdem_v2x_neopat            15145
vdem_v2x_pubcorr           15145
vdem_v2x_rule              15145
vdem_v2x_veracc            15145
vdem_v2x_ex_military       15145
vdem_v2x_ex_party          15145
vdem_v2x_freexp            15145
vdem_v2xcl_acjst           15145
vdem_v2xcl_dmove           15145
vd

In [14]:
# Loop that checks whether the model exists, retrains if not, 
# and stores the predictions if they have not been stored before for this run.
# To do: set the data_preprocessing to the function in the model dictionary

level = 'cm'
includeFuture = False
force_rewrite = True
force_retrain = True

from views_runs import Storage, StepshiftedModels
from views_partitioning.data_partitioner import DataPartitioner
from viewser import Queryset, Column
from views_runs import operations
from views_runs.run_result import RunResult
from new_markov import markov

i = 0
for model in ModelList[:]:
    if 'Markov' not in model['modelname']:
        
        modelstore = storage.Storage()
        ct = datetime.now()
        print(i, model['modelname'])
        print('Calibration partition', ct)
        model['Algorithm_text'] = str(model['algorithm'])
        model['RunResult_calib'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"calib":calib_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "calib",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_calib',
                author_name        = "JED",
        )

    #    model['predstore_calib'] = level +  '_' + model['modelname'] + '_calib'
        ct = datetime.now()
        if force_rewrite:
            print(model['predstore_calib'], ', run',  run_id, 'force_rewrite=True, predicting')
            predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)
            predictions_calib.forecasts.set_run(run_id)
            predictions_calib.forecasts.to_store(name=model['predstore_calib'],overwrite=True)
        else:
            print('Trying to retrieve predictions', ct)
            try:
                predictions_calib = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_calib'])
            except KeyError:
                print(model['predstore_calib'], ', run',  run_id, 'does not exist, predicting')
                predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)
                predictions_calib.forecasts.set_run(run_id)
                predictions_calib.forecasts.to_store(name=model['predstore_calib'])

        ct = datetime.now()
        print('Test partition', ct)
        modelstore = storage.Storage()
        model['RunResult_test'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"test":test_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "test",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_test',
                author_name        = "JED",
        )
        ct = datetime.now()
        
        if force_rewrite:
            print(model['predstore_test'], ', run',  run_id, 'force_rewrite=True, predicting')
            predictions_test = model['RunResult_test'].run.predict("test","predict", model['RunResult_test'].data)
            predictions_test.forecasts.set_run(run_id)
            predictions_test.forecasts.to_store(name=model['predstore_test'],overwrite=True)
        else:
            print('Trying to retrieve predictions', ct)
    #    model['predstore_test'] = level +  '_' + model['modelname'] + '_test'
            try:
                predictions_test = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_test'])
            except KeyError:
                print(model['predstore_test'], ', run', run_id, 'does not exist, predicting')
                predictions_test = model['RunResult_test'].run.predict("test","predict",model['RunResult_test'].data)
                predictions_test.forecasts.set_run(run_id)
                predictions_test.forecasts.to_store(name=model['predstore_test'])
        # Predictions for true future
        if includeFuture:
            ct = datetime.now()
            print('Future', ct)
            modelstore = storage.Storage()
            model['RunResult_future'] = RunResult.retrain_or_retrieve(
                    retrain            = force_retrain,
                    store              = modelstore,
                    partitioner        = DataPartitioner({"test":future_partitioner_dict}),
                    stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                    dataset            = RetrieveFromList(Datasets,model['data_train']),
                    queryset_name      = model['queryset'],
                    partition_name     = "test",
                    timespan_name      = "train",
                    storage_name       = model['modelname'] + '_future',
                    author_name        = "JED",
            )
            ct = datetime.now()
            
            if force_rewrite:
                print(model['predstore_future'], ', run',  run_id, 'force_rewrite=True, predicting')
                predictions_future = model['RunResult_future'].run.predict(EndOfHistory, model['RunResult_future'].data)
                predictions_future.forecasts.set_run(run_id)
                predictions_future.forecasts.to_store(name=model['predstore_future'],overwrite=True)
            else:
                print('Trying to retrieve predictions', ct)
                model['predstore_future'] = level +  '_' + model['modelname'] + '_f' + str(EndOfHistory)
                predictions_future.forecasts.to_store(name=model['predstore_future'])  
                
    else:
        modelstore = storage.Storage()
        ct = datetime.now()
        print(i, model['modelname'])
        print('Calibration partition', ct)
        model['Algorithm_text'] = str(model['algorithm'])
        print('Trying to retrieve predictions', ct)
        if force_retrain:
            print(model['predstore_calib'], ', run',  run_id, 'force_retrain = True, predicting')
            predictions_calib = markov.compute_markov(calib_partitioner_dict, EndOfHistory, model['depvar'], 'calib', model['algorithm'])
            predictions_calib.forecasts.set_run(run_id)
            predictions_calib.forecasts.to_store(name=model['predstore_calib'],overwrite=True)
        else:
            try:
                predictions_calib = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_calib'])
            except KeyError:
                print(model['predstore_calib'], ', run',  run_id, 'does not exist, predicting')
                predictions_calib = markov.compute_markov(calib_partitioner_dict, EndOfHistory, model['depvar'], 'calib', model['algorithm'])
                predictions_calib.forecasts.set_run(run_id)
                predictions_calib.forecasts.to_store(name=model['predstore_calib'],overwrite=True)
                
        ct = datetime.now()
        print('Test partition', ct)
        modelstore = storage.Storage()
        if force_retrain:
            print(model['predstore_test'], ', run', run_id, 'force_retrain=True, predicting')
            predictions_test = markov.compute_markov(test_partitioner_dict, EndOfHistory, model['depvar'], 'test', model['algorithm'])
            predictions_test.forecasts.set_run(run_id)
            predictions_test.forecasts.to_store(name=model['predstore_test'],overwrite=True)
        else:
            try:
                predictions_test = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_test'])
            except KeyError:
                print(model['predstore_test'], ', run', run_id, 'does not exist, predicting')
                predictions_test = markov.compute_markov(test_partitioner_dict, EndOfHistory, model['depvar'], 'test', model['algorithm'])
                predictions_test.forecasts.set_run(run_id)
                predictions_test.forecasts.to_store(name=model['predstore_test'],overwrite=True)
                
        if includeFuture:
            ct = datetime.now()
            print('Future', ct)
            modelstore = storage.Storage()
            print('Trying to retrieve predictions', ct)
            model['predstore_future'] = level +  '_' + model['modelname'] + '_f' + str(EndOfHistory)
            if force_retrain:
                print(model['predstore_future'], ', run', run_id, 'force_retrain=True, predicting')
                predictions_future = markov.compute_markov(test_partitioner_dict, EndOfHistory, model['depvar'], 'future', model['algorithm'])
                predictions_future.forecasts.set_run(run_id)
                predictions_future.forecasts.to_store(name=model['predstore_future'],overwrite=True)
            else:
                try:
                    predictions_future = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_future'])
                except KeyError:
                    print(model['predstore_future'], ', run', run_id, 'does not exist, predicting')
                    predictions_future = markov.compute_markov(test_partitioner_dict, EndOfHistory, model['depvar'], 'future', model['algorithm'])
                    predictions_future.forecasts.set_run(run_id)
                    predictions_future.forecasts.to_store(name=model['predstore_future'],overwrite=True)  
                            
        print('**************************************************************')
    i = i + 1

print('All done')

0 escwa001_cflong
Calibration partition 2023-05-03 21:23:21.447161
 * == Performing a run: "escwa001_cflong_calib" == * 
Model object named "escwa001_cflong_calib" with equivalent metadata already exists.
Retrain is true, overwriting "escwa001_cflong_calib"
Training model(s)...
Storing "escwa001_cflong_calib"
cm_escwa001_cflong_calib , run escwa001 force_rewrite=True, predicting
Test partition 2023-05-03 21:25:32.414794
 * == Performing a run: "escwa001_cflong_test" == * 
Model object named "escwa001_cflong_test" with equivalent metadata already exists.
Retrain is true, overwriting "escwa001_cflong_test"
Training model(s)...
Storing "escwa001_cflong_test"
cm_escwa001_cflong_test , run escwa001 force_rewrite=True, predicting
1 escwa001_vdem
Calibration partition 2023-05-03 21:28:13.658211
 * == Performing a run: "escwa001_vdem_calib" == * 
Training model(s)...
Storing "escwa001_vdem_calib"
cm_escwa001_vdem_calib , run escwa001 force_rewrite=True, predicting
Test partition 2023-05-03 21:

## Notes on training time for the various algorithms:

In [ ]:
#These are calculated in minutes for the hh20 feature set (with about 40 features), for all 36 steps, calibration (c) and test (t) partitions, also include generating predictions, and are approximate:

#nj=12 (number of threads)
#scikit random forest:        21:13 (c), 26:20 (t) RandomForestRegressor(n_estimators=200, n_jobs=nj)
#XGB random forest:           06:02 (c), 07:51 (t) XGBRFRegressor(n_estimators=300,n_jobs=nj)
#scikit gbm:                  13:59 (c), 15:55 (t) GradientBoostingRegressor(), 
#scikit hurdle random forest: 07:32 (c), 09:49 (t) For both clf and reg: (n_estimators=200, n_jobs=nj)
#XGB hurdle xgb:              01:26 (c), 01:32 (t) For both clf and reg:                n_estimators=200,tree_method='hist',n_jobs=nj)
#scikit histgbm:              01:17 (c), 01:20 (t) HistGradientBoostingRegressor(max_iter=200)
#XGB xgb:                     01:00 (c), 01:04 (t) XGBRegressor(n_estimators=200,tree_method='hist',n_jobs=nj)
#lightgbm gbm:                00:25 (c), --    (t) LGBMRegressor(n_estimators=100,num_threads=8)

# Various helper functions and tools....

# Retrieving external forecasts

In [ ]:
## Retrieve David's Markov models
## To do: rewrite the model dictionary to the new, slimmer version.
#DRList = []


#model = {
#    'modelname':   'fatalities002_Markov_glm',
#    'algorithm': [],
#    'depvar': "ln_ged_sb_dep",
#    'data_train':      'joint_narrow',
#    'queryset': 'fatalities002_joint_narrow',
#}
#DRList.append(model)

#model = {
#    'modelname':   'fatalities002_Markov_rf',
#    'algorithm': [],
#    'depvar': "ln_ged_sb_dep",
#    'data_train':      'joint_narrow',
#    'queryset': 'fatalities002_joint_narrow',
#}

#DRList.append(model)



In [ ]:
#path = f'/Users/{os.getlogin()}/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/Predictions/cm/preds/'

#DRList[0]['predictions_file_calib'] = path + 'markov_jointnarrow_ss_glm_calib.parquet'
#DRList[0]['predictions_file_test'] = path + 'markov_jointnarrow_ss_glm_test.parquet'
#DRList[0]['predictions_file_future'] = path + 'vmm_glm_hh20_517.csv'

#DRList[1]['predictions_file_calib'] = path + 'markov_jointnarrow_ss_rf_calib.parquet'
#DRList[1]['predictions_file_test'] = path + 'markov_jointnarrow_ss_rf_test.parquet'
#DRList[1]['predictions_file_future'] = path + 'vmm_rf_hh20_517.csv'

In [ ]:
## Storing Markov models in central storage
## Retrieving dependent variable

#print('Adding depvar - CHECK FILES BEING USED FROM STORAGE ARE SUITABLE!')
#target_calib = pd.DataFrame.forecasts.read_store('cm_fatalities002_conflicthistory_rf_calib', run=run_id)['ln_ged_sb_dep']
#target_test = pd.DataFrame.forecasts.read_store('cm_fatalities002_conflicthistory_rf_test', run=run_id)['ln_ged_sb_dep']
#level = 'cm'
#for model in DRList:
#    df_calib = pd.read_parquet(model['predictions_file_calib'])
##    df_calib.rename(columns={'target_month_id':'month_id'}, inplace=True)
##    df_calib.set_index(['month_id', 'country_id'], inplace=True)

#    df_test = pd.read_parquet(model['predictions_file_test'])
##    df_test.rename(columns={'target_month_id':'month_id'}, inplace=True)
##    df_calib.set_index(['month_id', 'country_id'], inplace=True)

##    df_future = pd.read_csv(model['predictions_file_future'],index_col=['month_id','country_id'])
#    df_calib['ln_ged_sb_dep'] = target_calib
#    df_test['ln_ged_sb_dep'] = target_test
##    df_future['ln_ged_sb_dep'] = np.nan # Empty dependent variable column for consistency/required by prediction storage function
#    stored_modelname = level + '_' + model['modelname'] + '_calib'
#    df_calib.forecasts.set_run(run_id)
#    df_calib.forecasts.to_store(name=stored_modelname, overwrite=True)
#    stored_modelname = level + '_' + model['modelname'] + '_test'
#    df_test.forecasts.set_run(run_id)
#    df_test.forecasts.to_store(name=stored_modelname, overwrite=True)    